<a href="https://colab.research.google.com/github/nhut-ngnn/Practice-Graph_Neural_Network/blob/main/GNN3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 43.4 MB/s eta 0:00:00


In [6]:
import torch
from torch_geometric.data import Data
from itertools import combinations

x = torch.tensor([
    [1.0, 2.0, 3.0, 4.0, 5.0, 6.0],
    [2.0, 3.0, 4.0, 5.0, 6.0, 7.0],
    [3.0, 4.0, 5.0, 6.0, 7.0, 8.0],
    [4.0, 5.0, 6.0, 7.0, 8.0, 9.0],
    [5.0, 6.0, 7.0, 8.0, 9.0, 10.0]
], dtype=torch.float)

edge_index = torch.tensor(list(combinations(range(x.size(0)), 2)), dtype=torch.long).t().contiguous()

y = torch.tensor([0, 1, 0, 1, 0], dtype=torch.long)

train_mask = torch.tensor([1, 1, 0, 0, 1], dtype=torch.bool)
test_mask = torch.tensor([0, 0, 1, 1, 0], dtype=torch.bool)

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

print(f"Number of nodes: {data.num_nodes}")
print(f"Number of edges: {data.num_edges}")
print(f"Node feature matrix shape: {data.x.shape}")


Number of nodes: 5
Number of edges: 10
Node feature matrix shape: torch.Size([5, 6])


In [7]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, num_node_features, hidden_channels, num_classes):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(num_node_features, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, num_classes)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)

        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)


In [8]:
model = GCN(num_node_features=data.num_node_features, hidden_channels=16, num_classes=2)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(200):
    optimizer.zero_grad()
    out = model(data)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()

    if epoch % 20 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')


Epoch 0, Loss: 0.8140568137168884
Epoch 20, Loss: 2.6332709789276123
Epoch 40, Loss: 0.8695505261421204
Epoch 60, Loss: 0.8488184809684753
Epoch 80, Loss: 0.6004019975662231
Epoch 100, Loss: 0.6730719208717346
Epoch 120, Loss: 0.6526309847831726
Epoch 140, Loss: 0.5942161679267883
Epoch 160, Loss: 0.6227142214775085
Epoch 180, Loss: 0.6446627974510193


In [9]:
model.eval()
_, pred = model(data).max(dim=1)
correct = pred[data.test_mask].eq(data.y[data.test_mask]).sum().item()
acc = correct / data.test_mask.sum().item()
print(f'Accuracy: {acc:.4f}')


Accuracy: 0.5000
